In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("results_reduces.csv", delimiter=";")
df

,ID,openmlid,seed,max_diff,iterations_with_max_difff,creation_date,status,start_date,name,machine,scores,end_date,error
0,1827,40975,6,0.0001,1000,"10/11/2022, 23:21:22",done,"10/12/2022, 01:29:54",b'PyExperimenter',n2cn0192,"b'[[0, 2, 0, 0.8102, 0.4329, 0.0], [3, 2, 0, 0...","10/12/2022, 01:30:57",NaN


In [3]:
openmlid = 40975
seed = 6
row = df[(df["openmlid"] == openmlid) & (df["seed"] == seed)]["scores"].values[0].replace("b'", "").replace("'", "")
input_history = json.loads(row)
input_history

[[0, 2, 0, 0.8102, 0.4329, 0.0],
 [3, 2, 0, 0.6564, 0.2978, 0.0],
 [6, 2, 0, 0.5635, 0.2237, 0.0],
 [8, 2, 0, 0.504, 0.2013, 0.0],
 [10, 2, 0, 0.466, 0.1905, 0.0],
 [13, 2, 0, 0.4373, 0.1758, 0.0],
 [15, 2, 0, 0.4145, 0.168, 0.0],
 [18, 2, 0, 0.3964, 0.1642, 0.0],
 [21, 2, 0, 0.3814, 0.1588, 0.0],
 [23, 2, 0, 0.3664, 0.1507, 0.0],
 [25, 2, 0, 0.3553, 0.1485, 0.0],
 [28, 2, 0, 0.3441, 0.1466, 0.0],
 [30, 2, 0, 0.336, 0.1439, 0.0],
 [33, 2, 0, 0.3297, 0.1489, 0.0],
 [35, 2, 0, 0.3238, 0.1499, 0.0],
 [38, 2, 0, 0.3175, 0.1497, 0.0],
 [40, 2, 0, 0.3116, 0.1506, 0.0],
 [43, 2, 0, 0.3058, 0.1488, 0.0],
 [45, 2, 0, 0.3024, 0.1475, 0.0],
 [48, 2, 0, 0.2973, 0.145, 0.0],
 [50, 2, 0, 0.293, 0.1448, 0.0],
 [53, 2, 0, 0.2891, 0.1432, 0.0],
 [56, 2, 0, 0.2858, 0.1419, 0.0],
 [58, 2, 0, 0.282, 0.1396, 0.0],
 [61, 3, 0, 0.2794, 0.1397, 0.0],
 [64, 2, 0, 0.2763, 0.1383, 0.0],
 [67, 2, 0, 0.2738, 0.1372, 0.0],
 [69, 2, 0, 0.2712, 0.1357, 0.0],
 [72, 2, 0, 0.2689, 0.1353, 0.0],
 [74, 2, 0, 0.2672, 0.135

In [4]:
import logging
logger = logging.getLogger("exp")
logger.setLevel(logging.WARN)

In [21]:
from asforests._grower import *
import time
import matplotlib.pyplot as plt

def get_gap_to_final_score(input_history, openmlid, seed, w_min, epsilon, extrapolation_multiplier, delta, bootstrap_repeats):

    # create info supplier
    input_scores = [e[3] for e in input_history]
    info_supplier = get_dummy_info_supplier(input_scores)
    final_score = input_scores[-1]

    step_size = min(w_min - 1, int(extrapolation_multiplier / 10**3))
    print(step_size)
    
    # now run algorithm
    start = time.time()
    grower = ForestGrower(info_supplier, d = 1, step_size = step_size, w_min = w_min, epsilon = epsilon, extrapolation_multiplier = extrapolation_multiplier, delta = delta, max_trees = np.inf, random_state = seed, stop_when_horizontal = True, bootstrap_repeats = bootstrap_repeats, logger = logger)
    try:
        grower.grow()
    except StopIteration:
        pass
    
    end = time.time()
    if False:
        fig, axs = plt.subplots(1, 2, figsize=(14, 3))
        axs[0].scatter(range(len(grower.time_stats_cauchy)), grower.time_stats_cauchy)
        axs[1].scatter(range(len(grower.time_stats_slope)), grower.time_stats_slope)
        plt.show()
        print(np.sum(grower.time_stats_steps))
        print(np.sum(grower.time_stats_supplier))
        print(np.sum(grower.time_stats_preamble))
        print(np.sum(grower.time_stats_cauchy))
        print(np.sum(grower.time_stats_slope))
        print(np.sum(grower.cov_times))
    output_history = grower.histories[0]
    runtime = np.round((end - start) * 1000)
    return len(output_history), output_history[-1] - final_score, runtime

In [32]:
%%time
w_min = 1000
epsilon = 0.001
c = 10**4
delta = 1000
bootstrap_repeats = 20

#times = []
#for i in range(20):
#    start = time.time()
get_gap_to_final_score(input_history, openmlid, seed, w_min, epsilon, c, delta, bootstrap_repeats)
 #   times.append(time.time() - start)
    #print(times[-1])

10
CPU times: user 10.1 s, sys: 233 ms, total: 10.3 s
Wall time: 2.62 s


(10180, 0.0009000000000000119, 2619.0)

In [7]:
times_after = times.copy()

NameError: name 'times' is not defined

In [ ]:
fig, ax = plt.subplots()
ax.boxplot([times_before, times_after])